In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/Agriculture/Data'
train = pd.read_csv(path + "/train.csv")
test = pd.read_csv(path + "/test.csv")
# submission = pd.read_csv(path + "/sample_submission.csv")

In [ ]:
train.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,F00000001,188,1,0,1,0,0.0,0,1,0
1,F00000003,209,1,0,1,0,0.0,0,2,1
2,F00000004,257,1,0,1,0,0.0,0,2,1
3,F00000005,257,1,1,1,0,0.0,0,2,1
4,F00000006,342,1,0,1,0,0.0,0,2,1


### Filling the null values in Number_Weeks_Used column

In [ ]:
train['Number_Weeks_Used'] = train['Number_Weeks_Used'].fillna(
                            train.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].transform('median'))
test['Number_Weeks_Used'] = test['Number_Weeks_Used'].fillna(
                            test.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].transform('median'))

### Data Preprocessing

In [ ]:
training_labels = train.iloc[:, -1]
X_train = train.iloc[:, 1:-1]
X_test = test.iloc[:, 1:]

In [ ]:
data = pd.concat([X_train, X_test])

In [ ]:
data.head()

,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season
0,188,1,0,1,0,0.0,0,1
1,209,1,0,1,0,0.0,0,2
2,257,1,0,1,0,0.0,0,2
3,257,1,1,1,0,0.0,0,2
4,342,1,0,1,0,0.0,0,2


### OneHotEncoding for "Pesticide_Use_Category" and "Season"

In [ ]:
columns_names_encod = data.columns[[3, 7]]
data = pd.get_dummies(data, columns=columns_names_encod)

In [ ]:
data

,Estimated_Insects_Count,Crop_Type,Soil_Type,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Pesticide_Use_Category_1,Pesticide_Use_Category_2,Pesticide_Use_Category_3,Season_1,Season_2,Season_3
0,188,1,0,0,0.0,0,1,0,0,1,0,0
1,209,1,0,0,0.0,0,1,0,0,0,1,0
2,257,1,0,0,0.0,0,1,0,0,0,1,0
3,257,1,1,0,0.0,0,1,0,0,0,1,0
4,342,1,0,0,0.0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
59305,3337,1,0,20,34.0,12,0,1,0,1,0,0
59306,3516,1,0,20,32.0,10,0,1,0,0,1,0
59307,3702,1,0,10,26.0,48,0,1,0,1,0,0
59308,3702,1,0,10,28.0,17,0,1,0,0,1,0


### Splitting the data back

In [ ]:
final_train = data[:train.shape[0]]
final_test = data[train.shape[0]:]

In [ ]:
final_train

,Estimated_Insects_Count,Crop_Type,Soil_Type,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Pesticide_Use_Category_1,Pesticide_Use_Category_2,Pesticide_Use_Category_3,Season_1,Season_2,Season_3
0,188,1,0,0,0.0,0,1,0,0,1,0,0
1,209,1,0,0,0.0,0,1,0,0,0,1,0
2,257,1,0,0,0.0,0,1,0,0,0,1,0
3,257,1,1,0,0.0,0,1,0,0,0,1,0
4,342,1,0,0,0.0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
88853,3337,1,0,10,12.0,44,0,1,0,0,0,1
88854,3516,1,0,10,20.0,38,0,1,0,1,0,0
88855,3516,1,0,15,40.0,8,0,1,0,0,1,0
88856,3702,1,0,10,25.0,18,0,1,0,0,0,1


In [ ]:
final_test

,Estimated_Insects_Count,Crop_Type,Soil_Type,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Pesticide_Use_Category_1,Pesticide_Use_Category_2,Pesticide_Use_Category_3,Season_1,Season_2,Season_3
0,188,1,1,0,0.0,0,1,0,0,0,1,0
1,410,1,1,0,0.0,0,1,0,0,0,1,0
2,626,1,0,0,0.0,0,1,0,0,0,1,0
3,731,1,0,0,0.0,0,1,0,0,0,1,0
4,789,0,0,0,0.0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
59305,3337,1,0,20,34.0,12,0,1,0,1,0,0
59306,3516,1,0,20,32.0,10,0,1,0,0,1,0
59307,3702,1,0,10,26.0,48,0,1,0,1,0,0
59308,3702,1,0,10,28.0,17,0,1,0,0,1,0


### Creating cross validation set

In [ ]:
training_labels.value_counts()

0    74238
1    12307
2     2313
Name: Crop_Damage, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_training, X_cv, y_training, y_cv = train_test_split(final_train, training_labels, test_size =0.2, random_state=21)

In [ ]:
from collections import Counter
Counter(y_training)

Counter({0: 59448, 1: 9783, 2: 1855})

## Training model on training dataset

In [ ]:
# To check whether GPU is running or not
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# lightgbm for classification
from numpy import mean
from numpy import std
#from sklearn.datasets import make_classification
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
#from matplotlib import pyplot
# define dataset
#X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=1)

# evaluate the model
model = LGBMClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_training, y_training, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# fit the model on the whole dataset
model = LGBMClassifier()
model.fit(X_training, y_training)

y_training_pred = model.predict(X_training)
y_cv_pred = model.predict(X_cv)

Accuracy: 0.847 (0.002)


### Checking accuracy score for training dataset and cross validation set

In [ ]:
from sklearn.metrics import accuracy_score
training_accuracy_score = accuracy_score(y_training, y_training_pred)
cv_accuracy_score = accuracy_score(y_cv, y_cv_pred)

print(training_accuracy_score)
print(cv_accuracy_score)

0.851109923191627
0.8445307224848075


### Testing the model on test data

In [ ]:
p = model.predict(final_train)
p = pd.Series(p)
Counter(p)

Counter({0: 84745, 1: 4113})

In [ ]:
predictions_test = model.predict(final_test)
predictions_test = pd.Series(predictions_test)

frame = {'ID': test.ID, 
         'Crop_Damage': predictions_test}
submission = pd.DataFrame(frame)
submission.to_csv(path + '/lgbm_submission.csv', index=False)

In [ ]:
final_test

,Estimated_Insects_Count,Crop_Type,Soil_Type,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Pesticide_Use_Category_1,Pesticide_Use_Category_2,Pesticide_Use_Category_3,Season_1,Season_2,Season_3
0,188,1,1,0,0.0,0,1,0,0,0,1,0
1,410,1,1,0,0.0,0,1,0,0,0,1,0
2,626,1,0,0,0.0,0,1,0,0,0,1,0
3,731,1,0,0,0.0,0,1,0,0,0,1,0
4,789,0,0,0,0.0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
59305,3337,1,0,20,34.0,12,0,1,0,1,0,0
59306,3516,1,0,20,32.0,10,0,1,0,0,1,0
59307,3702,1,0,10,26.0,48,0,1,0,1,0,0
59308,3702,1,0,10,28.0,17,0,1,0,0,1,0


In [ ]:
predictions_test

0        1
1        1
2        1
3        1
4        1
        ..
59305    0
59306    0
59307    0
59308    0
59309    0
Length: 59310, dtype: int64

In [ ]:
import joblib

In [ ]:
joblib.dump(model,'/content/drive/MyDrive/Colab Notebooks/Agriculture/lgbm84.ml')

['/content/drive/MyDrive/Colab Notebooks/Agriculture/lgbm84.ml']